In [19]:
import numpy as np
import keras

from typing import Callable


In [25]:
def relu(x: np.ndarray) -> float | np.ndarray:
    return np.max(0, x)


def sigmoid(x: np.ndarray) -> float | np.ndarray:
    return 1 / (1 + np.exp(-x))


In [82]:
class Layer:

    def __init__(self, shape:list[int, int] | int, initial_value: float = 0.0) -> None:

        self.__shape = shape

        self.__values: np.ndarray = np.zeros(
            shape = self.__shape,
            dtype = np.float32
        ) + initial_value

        self.__weights: np.ndarray = None

    @property
    def shape(self) -> list[int]:
        return self.__shape

In [83]:
class Dense(Layer):

    def __init__(self, units: int, initial_value: float = 0.0, activation: str | Callable = "relu") -> np.ndarray:
        super().__init__(units, initial_value)

        self.__activation: Callable = self.set_activation(activation)

    def set_activation(self, activation: str | Callable) -> None:
        if isinstance(activation, str):
            match activation.lower():
                case "relu":
                    self.__activation = relu
                case "sigmoid":
                    self.__activation = sigmoid
                case "linear":
                    self.__activation = lambda x: x
                case _:
                    raise ValueError(
                        f"Dense layer does not support activation function: {activation}")
        elif callable(activation):
            self.__activation = activation
        else:
            raise ValueError(
                "Argument `activation` must be of type `str` or `function`.")

    def update(self, values: np.ndarray) -> None:
        self.__values = self.__activation(values)

    def set_weights(self, shape: list[int, int], initial_value: float = 0.0) -> None:
        self.__weights = np.zeros(
            shape,
            dtype = np.float32
        ) + initial_value

    @property
    def info(self):
        if self.__weights is None:
            weight_shape: int  = 0
        else:
            weight_shape: list[int, int] = self.__weights.shape

        return f"Dense Layer with {self.shape} units and weight matrix of shape {weight_shape}"


In [84]:
class FFNN:
    """Naive implementation of a feed-forward multi-layer perceptron (neural network)"""
    
    def __init__(self, layers: Layer) -> None:
        self.__layers: list[Layer] = layers
        
        self.initialize_weights()

    def initialize_weights(self):
        for i in range(1, len(self.__layers)):
            prev_layer: Layer = self.__layers[i - 1]
            curr_layer: Layer = self.__layers[i]

            shape = [prev_layer.shape, curr_layer.shape]
            prev_layer.set_weights(shape)

    @property
    def info(self):
        for layer in self.__layers:
            print(layer.info)

In [85]:
FFNN([
    Dense(10, activation="linear"),
    Dense(10, activation="relu"),
    Dense(10, activation="relu")
]).info

Dense Layer with 10 units and weight matrix of shape (10, 10)
Dense Layer with 10 units and weight matrix of shape (10, 10)


AttributeError: 'Dense' object has no attribute '_Dense__weights'